# Module

In [1]:
from glob import glob
import pandas as pd
import numpy as np
from tqdm import tqdm

# Data preprocessing

In [2]:
# train.csv path
df = pd.read_csv('/data/ephemeral/home/mask_data/train/train.csv')
df.head()

,id,gender,race,age,path
0,000001,female,Asian,45,000001_female_Asian_45
1,000002,female,Asian,52,000002_female_Asian_52
2,000004,male,Asian,54,000004_male_Asian_54
3,000005,female,Asian,58,000005_female_Asian_58
4,000006,female,Asian,59,000006_female_Asian_59


In [3]:
# Image path
data_path = '/data/ephemeral/home/mask_data/train/images/'
data_list = sorted(glob(data_path + '*'))
data_list[:5]

['/data/ephemeral/home/mask_data/train/images/000001_female_Asian_45',
 '/data/ephemeral/home/mask_data/train/images/000002_female_Asian_52',
 '/data/ephemeral/home/mask_data/train/images/000004_male_Asian_54',
 '/data/ephemeral/home/mask_data/train/images/000005_female_Asian_58',
 '/data/ephemeral/home/mask_data/train/images/000006_female_Asian_59']

In [4]:
new_data = []

for i in tqdm(range(len(data_list))):

  # 성별과 나이를 가져옵니다
  sex, age = data_list[i].split('/')[-1].split('_')[1],int(data_list[i].split('/')[-1].split('_')[-1])
  imgs_path = glob(data_list[i]+'/*')

  # 마스크 정보 받아오기
  labels = []

  for p in imgs_path:
    label = p.split('/')[-1][:-4]
    new_data.append([p, sex, age, label])


100%|██████████| 2700/2700 [00:00<00:00, 20585.69it/s]


In [5]:
# 정답 정보를 가진 새로운 dataframe 생성
new_df = pd.DataFrame(new_data, columns=['path', 'gender', 'age', 'mask'])
new_df.head()

,path,gender,age,mask
0,/data/ephemeral/home/mask_data/train/images/00...,female,45,normal
1,/data/ephemeral/home/mask_data/train/images/00...,female,45,mask1
2,/data/ephemeral/home/mask_data/train/images/00...,female,45,mask3
3,/data/ephemeral/home/mask_data/train/images/00...,female,45,mask2
4,/data/ephemeral/home/mask_data/train/images/00...,female,45,mask4


# Holdout

In [6]:
from sklearn.model_selection import train_test_split
from PIL import Image

In [7]:
X_train, X_valid, y_train, y_valid = train_test_split(new_df['path'].values, new_df['age'].values, test_size=0.3, stratify=new_df['age'])
print('Shape of X train : ',X_train.shape)
print('Shape of y train : ',y_train.shape)
print('Shape of X valid : ',X_valid.shape)
print('Shape of y valid : ',y_valid.shape)

Shape of X train :  (13230,)
Shape of y train :  (13230,)
Shape of X valid :  (5670,)
Shape of y valid :  (5670,)


# Dataset

In [8]:
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms
from albumentations import *
import matplotlib.pyplot as plt
from albumentations.pytorch import ToTensorV2

In [9]:
def get_transforms(need=('train', 'val'), img_size=(224, 224), mean=(0.548, 0.504, 0.479), std=(0.237, 0.247, 0.246)):
    """
    train 혹은 validation의 augmentation 함수를 정의합니다. train은 데이터에 많은 변형을 주어야하지만, validation에는 최소한의 전처리만 주어져야합니다.

    Args:
        need: 'train', 혹은 'val' 혹은 둘 다에 대한 augmentation 함수를 얻을 건지에 대한 옵션입니다.
        img_size: Augmentation 이후 얻을 이미지 사이즈입니다.
        mean: 이미지를 Normalize할 때 사용될 RGB 평균값입니다.
        std: 이미지를 Normalize할 때 사용될 RGB 표준편차입니다.

    Returns:
        transformations: Augmentation 함수들이 저장된 dictionary 입니다. transformations['train']은 train 데이터에 대한 augmentation 함수가 있습니다.
    """
    transformations = {}
    if 'train' in need:
        transformations['train'] = Compose([
            Normalize(mean=mean, std=std, max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)
    if 'val' in need:
        transformations['val'] = Compose([
            Normalize(mean=mean, std=std, max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)
    return transformations

In [10]:
class MaskDataset(Dataset):
    def __init__(self, path,y,transform,train):
        self.path = path
        self.y = y
        self.transforms = transform
        self.train = train

    def __len__(self):
        return len(self.path)

    def __getitem__(self,idx):
        img_path = self.path[idx]
        img = Image.open(img_path).convert(mode='RGB')
        label = torch.tensor(self.y[idx]).float()
        if self.transforms:
            if self.train:
              img = self.transforms['train'](image=np.array(img))['image']
            else:
              img = self.transforms['val'](image=np.array(img))['image']
 

        if self.train:
            return img, label
        else:
            return img_path, img, label

In [11]:
train_transform = get_transforms(need=('train'))
val_transform = get_transforms(need=('val'))

trainset = MaskDataset(X_train,y_train,train_transform,True)
validset = MaskDataset(X_valid, y_valid, val_transform, False)

In [12]:
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
validloader = DataLoader(validset, batch_size=16, shuffle=False)

# Model

In [13]:
from torchvision.models import resnet50
from torch import nn
import torch

In [14]:
# 모형 생성
resnet = resnet50(weights='IMAGENET1K_V1')
resnet.fc = nn.Linear(2048,1)

In [15]:
# 장치 설정
device = torch.device('cuda')
resnet.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [16]:
# 손실 함수 및 optimizer 정의
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(resnet.parameters(),lr=1e-3)
scaler = torch.cuda.amp.GradScaler()

# 학습

In [17]:
# Training
def train(epoch, model, dataloader, criterion, optimizer):
    model.train()
    train_loss = 0

    for batch_idx, (inputs, labels) in enumerate(dataloader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        outputs = outputs.squeeze(-1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
       

    epoch_loss = train_loss/len(dataloader)

    print("Train | Loss:%.4f" % (epoch_loss))
    return epoch_loss

In [18]:
# 추론
def test(epoch, model, dataloader,criterion, optimizer):
    model.eval()
    test_loss = 0
    
    with torch.no_grad():
        for batch_idx, (_,inputs, labels) in enumerate(dataloader):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs).squeeze(-1)
            loss = criterion(outputs, labels)

            test_loss += loss.item()

    epoch_loss = test_loss/len(dataloader)

    print("Test | Loss:%.4f"% (epoch_loss))
    return epoch_loss

In [19]:
import time
import copy

start_time = time.time()
best_loss = 9999999
best_model_wts = copy.deepcopy(resnet.state_dict())
epoch_length = 20
patience = 3
i = 0
save_loss = {"train":[],
             "val":[]}
save_acc = {"train":[],
             "val":[]}
for epoch in range(epoch_length):
    print("Epoch %s" % epoch)
    train_loss = train(epoch, resnet,trainloader, loss_fn, optimizer)
    save_loss['train'].append(train_loss)

    val_loss = test(epoch, resnet,validloader,loss_fn, optimizer)
    save_loss['val'].append(val_loss)
   

    # Save model
    if val_loss <= best_loss:
        best_loss = val_loss
        best_model_wts = copy.deepcopy(resnet.state_dict())
        i = 0
    else:
        i += 1
        if i >= patience:
            print('Early Stopping')
            break

    learning_time = time.time() - start_time
    print(f'**Learning time: {learning_time // 60:.0f}m {learning_time % 60:.0f}s')

Epoch 0
Train | Loss:106.7511
Test | Loss:28.3035
**Learning time: 4m 48s
Epoch 1
Train | Loss:22.0845
Test | Loss:26.8784
**Learning time: 9m 33s
Epoch 2
Train | Loss:17.4656
Test | Loss:17.0209
**Learning time: 14m 18s
Epoch 3
Train | Loss:14.0444
Test | Loss:16.5787
**Learning time: 19m 3s
Epoch 4
Train | Loss:12.0911
Test | Loss:14.1852
**Learning time: 23m 48s
Epoch 5
Train | Loss:10.8935
Test | Loss:11.8765
**Learning time: 28m 33s
Epoch 6
Train | Loss:8.9772
Test | Loss:10.1717
**Learning time: 33m 19s
Epoch 7
Train | Loss:8.0149
Test | Loss:11.0340
**Learning time: 38m 4s
Epoch 8
Train | Loss:6.3726
Test | Loss:35.3950
**Learning time: 42m 50s
Epoch 9
Train | Loss:5.7847
Test | Loss:10.9526
Early Stopping


In [27]:
torch.save(best_model_wts,'ResNet50_Age_회귀.pth')

In [28]:
# 학습한 가중치 불러오기
resnet.load_state_dict(torch.load('ResNet50_Age_회귀.pth',map_location='cpu'))

<All keys matched successfully>

In [29]:
# Evaluation mode
resnet.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

# 추론

In [30]:
answer = []
preds = []

for _,x, y in tqdm(validloader):
    x = x.to(device)
    answer += y.tolist()
    y = y.to(device)
    with torch.no_grad():
        pred = resnet(x)
        preds += pred.tolist()

    

100%|██████████| 355/355 [00:57<00:00,  6.17it/s]


In [31]:
from sklearn.metrics import mean_squared_error

In [32]:
def RMSE(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [33]:
# 점수
rmse = RMSE(y_pred=preds,y_true=answer)
print('RMSE : ',rmse)

RMSE :  3.1908024966731303


# 60세를 몇 세로 추정하는지 확인해보기

In [38]:
preds = np.array(preds).reshape(-1).tolist()

In [39]:
age_df = pd.DataFrame({'answer':answer,'pred':preds})
age_df.tail()

,answer,pred
5665,57.0,53.090046
5666,57.0,53.782265
5667,20.0,21.158995
5668,56.0,55.187126
5669,58.0,58.079063


In [41]:
# 60세를 추정치
age_df[age_df.answer==60].pred.mean()

57.70488477758971

60세를 대략 57~58세로 추정하고 있음을 알 수 있다